In [2]:
# Imports
import numpy as np
from sympy import symbols, solve, Poly, simplify, latex, Rational, init_printing
import random
import cmath
import pandas as pd
from tqdm import tqdm

init_printing(use_latex=True)
from io import StringIO

---
# Problem type 1: Simple nondimensionalization of a random polynomial

For our first type of problem, we choose to do nondimensionalization problems of the format $a_1 x^{n_1} + a_2 x^{n_2} + a_3$ where $n_1 > n_2$


The expected form of the final answer will be $\epsilon y^{n_1} + y^{n_2} + 1$

In [3]:
def generate_n1n2(max_degree):
    """
    Generates two numbers n1, n2 < max_degree where n1 < n2
    """
    n2 = random.randint(1, max_degree - 1)
    n1 = random.randint(n2 + 1, max_degree)
    return n1, n2

In [12]:
def nondimensionalize_polynomial1(n1, n2):
    """
    Inputs:
    n1 (int): Degree of polynomial
    n2 (int): Second largest power of polynomial

    Outputs:
    question (string): Latex for problem
    answer (string): Latex for answer
    """
    # Define sympy symbols
    x, y, a1, a2, a3, epsilon = symbols('x y a_1 a_2 a_3 epsilon')

    # Generate sympy polynomial a1*x^n1 + a2*x^n2 + a3
    n1r, n2r = Rational(n1), Rational(n2)
    polynomial_x = a1 * x**n1r + a2 * x**n2r + a3 # CHANGE BACK TO PLUS

    # Define x = (a/b)y
    x_sub = ((a3/a2)**Rational(1/n2r)) * y

    # Substitute
    polynomial_y = polynomial_x.subs(x, x_sub).expand()
    # polynomial_y = polynomial_y.subs(x, y).expand()

    # Simplify before dividing by a3
    simplified_polynomial = polynomial_y / a3

    # Further simplify the expression
    final_simplified_polynomial = simplify(simplified_polynomial)

    # Return question and answer
    question = ""
    answer = ""

    question += "\nNondimensionalize the following polynomial"
    question += "\[" + latex(polynomial_x) + "\]"
    question += f"into one of the form $\epsilon y^{{{n1}}} + y^{{{n2}}} + 1$"

    answer += "\nStart with the following substitution:"
    answer += "\[" + "x=" + latex(x_sub) + "\]"

    answer += "\nThis gives us the following expression:"
    answer += "\[" + latex(polynomial_y) + "\]"

    answer += "\nDivide by the coefficient remaining in front of the constant, giving us the nondimensionalized polynomial with coefficients in terms of $a_1, a_2, a_3$:"
    answer += "\[ \\boxed{" + latex(final_simplified_polynomial) + "}\]"

    return question, answer

In [13]:
n1, n2 = generate_n1n2(max_degree=10)
question, answer = nondimensionalize_polynomial1(n1, n2)

question, answer = nondimensionalize_polynomial1(10, 4)

print(question)

print(answer)


Nondimensionalize the following polynomial\[a_{1} x^{10} + a_{2} x^{4} + a_{3}\]into one of the form $\epsilon y^{10} + y^{4} + 1$

Start with the following substitution:\[x=y \sqrt[4]{\frac{a_{3}}{a_{2}}}\]
This gives us the following expression:\[a_{1} y^{10} \left(\frac{a_{3}}{a_{2}}\right)^{\frac{5}{2}} + a_{3} y^{4} + a_{3}\]
Divide by the coefficient remaining in front of the constant, giving us the nondimensionalized polynomial with coefficients in terms of $a_1, a_2, a_3$:\[ \boxed{\frac{a_{1} y^{10} \left(\frac{a_{3}}{a_{2}}\right)^{\frac{5}{2}}}{a_{3}} + y^{4} + 1}\]


---
# Problem type 2: Nondimensionalization of polynomial with

The first problem type we have is to nondimensionalize a polynomial. There are two functions in this section:

1) Generate these random polynomials

2) Nondimensionalize these random polynomials

---
### Function to generate random polynomials
Generates a polynomial of up to a given degree with a specific number of terms. The coefficients of the polynomial are within the specified bounds.

In [74]:
def generate_polynomial(max_degree, num_terms, coeff_bounds):
    """
    Inputs:
    max_degree (int): The maximum degree of the polynomial.
    num_terms (int)
    coeff_bounds (tuple): (lower bound, upper bound)

    Outputs:
    coefficients (list)
    """
    # Initialize array of coeffs
    coefficients = [0] * (max_degree + 1)
    # Check number of terms
    if num_terms > max_degree + 1:
        raise ValueError("Number of terms cannot be more than the degree of the polynomial + 1")
    # Randomly choose positions for the (#) non-zero coefficients
    non_zero_positions = random.sample(range(max_degree), num_terms - 1)
    # Assign random values within the bounds to the chosen positions
    for pos in non_zero_positions:
        # Loop to make sure it is nonzero bc selecting from [-, +] includes 0
        while coefficients[pos] is None or coefficients[pos] == 0:
          coefficients[pos] = random.randint(coeff_bounds[0], coeff_bounds[1])
    # Ensure the highest degree term is non-zero
    if coefficients[-1] == 0:
        # While loop until it is nonzero
        while coefficients[-1] is None or coefficients[-1] == 0:
          coefficients[-1] = random.randint(coeff_bounds[0], coeff_bounds[1])

    return coefficients

In [104]:
# Test function
coefficients = generate_polynomial(max_degree=10, num_terms=3, coeff_bounds=[-10,10])
coefficients

[0, 0, 0, -2, 0, 0, 0, 0, 6, 0, 10]

---
### Helper function to generate sympy polynomial from coefficients

In [187]:
def sympy_polynomial_from_coefficients(coefficients):
    """
    Converts a list of coefficients into a polynomial expression using sympy.

    Parameters:
    coefficients (list): A list of coefficients, where the index represents the power of x.

    Returns:
    sympy expression: The polynomial expression.
    """
    x = symbols('x')
    coefficients_reverse = coefficients
    coefficients_reverse.reverse()
    polynomial = sum(coef * x**i for i, coef in enumerate(coefficients_reverse))
    return polynomial

In [188]:
sympy_polynomial_from_coefficients(coefficients)

     7      2     
- 2⋅x  + 6⋅x  + 10

---
### Nondimensionalization function

Takes in array of coefficients and returns in a nondimensionalized form (only works for 3-term polynomials due to how it reduces the 2nd and third terms to have coefficients of 1).

First converts it to the form  $\epsilon x^{n_1} \pm x^{n_2} \pm 1$ where $n_1 > n_2$

Finally, it calculates the value of epsilon and outputs the equivalent nondimensionalized polynomial.



In [221]:
def nondimensionalize_polynomial(coefficients):
    """
    Inputs:
    coefficients (list): The list of coefficients

    Outputs:
    s (string): Latex for problem and solution
    """
    question = ""
    answer = ""

    # Generate sympy polynomial from coefficients
    sympy_polynomial = sympy_polynomial_from_coefficients(coefficients)

    # Find nonzero coefficients & powers in the format [(coefficient, power)]
    nonzero_coeffs = [(coeff, len(coefficients) - idx - 1) for idx, coeff in enumerate(coefficients) if coeff != 0]
    # Ensure exactly 3 nonzero coefficients
    if len(nonzero_coeffs) != 3:
        return "Polynomial must have exactly three nonzero coefficients.", None

    # Find largest, second largest powers
    sorted_coeffs = sorted(nonzero_coeffs, key=lambda x: x[1], reverse=True)
    max_power = sorted_coeffs[0][1]
    second_max_power = sorted_coeffs[1][1]

    # Get values for coefficients
    a = [abs(sorted_coeffs[0][0]), abs(sorted_coeffs[1][0]), abs(sorted_coeffs[2][0])]
    a_signs = [np.sign(sorted_coeffs[0][0]), np.sign(sorted_coeffs[1][0]), np.sign(sorted_coeffs[2][0])]

    # Rename variables for largest, 2nd largest powers
    n1, n2 = Rational(max_power), Rational(second_max_power)

    # Define symbols for sympy
    x = symbols('x')
    a1, a2, a3 = symbols('a_1 a_2 a_3')
    epsilon = symbols('epsilon')

    # Define polynomial in sympy
    polynomial_x = a_signs[0]*(a1 * x**n1) + a_signs[1]*(a2 * x**n2) + a_signs[2]*a3

    # Define x = (a/b)y
    y = ((a3/a2)**(1/n2)) * x

    # Substitute
    polynomial_y = polynomial_x.subs(x, y)

    # Simplify before dividing by a3
    simplified_polynomial = polynomial_y / a3

    # Further simplify the expression
    final_simplified_polynomial = simplify(simplified_polynomial)

    # Substitute the values of a
    nondim_polynomial_subs_a = final_simplified_polynomial.subs([(a1,a[0]), (a2,a[1]), (a3,a[2])])

    # Get list of terms in polynomial
    nondim_poly_terms = list(nondim_polynomial_subs_a.expand().args)[::-1]
    # Divide by x term (x^n) to get the value of eps
    epsilon = nondim_poly_terms[0] / x**n1


    # Output results
    question += "\nNondimensionalize the following polynomial:"
    question += "\[" + latex(polynomial_x) + "\]"
# OR THIS PROBLEM - maybe separate into two types
    question += "\nNondimensionalize the following polynomial:"
    question += "\[" + latex(sympy_polynomial) + "\]"


    answer += "\nStart with the following substitution:"
    answer += "\[" + "y=" + latex(y) + "\]"

    answer += "\nThis gives us the following expression:"
    answer += "\[" + latex(polynomial_y) + "\]"

    answer += "\nDivide by the coefficient remaining in front of the constant, giving us the nondimensionalized polynomial with coefficients in terms of $a_1, a_2, a_3$:"
    answer += "\[" + latex(final_simplified_polynomial) + "\]"

    answer += "\nSubstitute in known values of $a_1, a_2, a_3$:"
    answer += "\[ \\boxed{" + latex(nondim_polynomial_subs_a) + "}\]"

    answer += "\nFrom the above equation we can identify the value of $\epsilon$ to be"
    answer += "\[" + latex(epsilon) + "\]"

    return question, answer

In [222]:
# Test function
question, answer = nondimensionalize_polynomial(coefficients)

# Print output
print(answer)


Start with the following substitution:\[y=x \sqrt{\frac{a_{3}}{a_{2}}}\]
This gives us the following expression:\[- a_{1} x^{7} \left(\frac{a_{3}}{a_{2}}\right)^{\frac{7}{2}} + a_{3} x^{2} + a_{3}\]
Divide by the coefficient remaining in front of the constant, giving us the nondimensionalized polynomial with coefficients in terms of $a_1, a_2, a_3$:\[- \frac{a_{1} x^{7} \left(\frac{a_{3}}{a_{2}}\right)^{\frac{7}{2}}}{a_{3}} + x^{2} + 1\]
Substitute in known values of $a_1, a_2, a_3$:\[ \boxed{- \frac{25 \sqrt{15} x^{7}}{81} + x^{2} + 1}\]
From the above equation we cn identify the value of $\epsilon$ to be\[- \frac{25 \sqrt{15}}{81}\]


### Save results to pandas dataframe, export to excel in desired format

In [200]:
# Number of desired problems
n_problems = 10

# Create empty dataframe
df = pd.DataFrame(columns=['Question', 'Answer'])

# Iterate once for each problem
for i in tqdm(range(n_problems)):
    # Generate new problem
    coefficients = generate_polynomial(max_degree=10, num_terms=3, coeff_bounds=[-10,10])
    # Solve new problem for new question, answer
    new_question, new_answer = nondimensionalize_polynomial(coefficients)
    # Add new question answer pair to data
    row_df = pd.DataFrame({'Question': [new_question], 'Answer': [new_answer]})
    df = pd.concat([df, row_df], ignore_index=True)

# Save dataframe to excel file of specified format
df.to_excel('polynomial_dataset.xlsx', index=False)

---
# Problem type 2: Solving for roots

Given a nondimensionalized problem of a certain form, find all of the roots in terms of epsilon.

---
### Generate nondimensionalized problem
Function for generating random nondimensionalized polynomial problem:
$\epsilon x^{n_1} \pm x^{n_2} \pm 1$ where $n_1 > n_2$

In [140]:
def generate_nondimensionalized(max_n1):
    """
    Inputs:
    max_n1 (int): Maximum power in polynomial

    Outputs:
    polynomial (sympy expression): Nondimensionalized random polynomial of the given format
    """
    # Ensure max_n1 is at least 2
    if max_n1 < 2:
        raise ValueError("max_n1 must be at least 2")

    # Randomly choose n1 and n2
    n1 = random.randint(2, max_n1)
    n2 = random.randint(1, n1 - 1)
    # Randomly choose signs
    signs = random.choices([-1, 1], k=2)

    # Construct polynomial
    x = symbols('x')
    epsilon = symbols('epsilon')
    polynomial = epsilon * x**n1 + signs[0] * x**n2 + signs[1]

    # Return
    return polynomial

In [141]:
# Example usage
polynomial = generate_nondimensionalized(10)
display(polynomial)

# Get terms of polynomial as a list
list(polynomial.expand().args)[::-1]

epsilon*x**9 + x**4 + 1

[epsilon*x**9, x**4, 1]

---
### Solve nondimensionalized problem

We use the following for dominant balances:


In [142]:
# Generate a polynomial
polynomial = generate_nondimensionalized(10)

In [143]:
# Extract terms and solve
x, epsilon = symbols('x epsilon')
terms = list(polynomial.expand().args)[::-1]
A, B, C = terms[0], terms[1], terms[2]
sol_ab = [simplify(sol) for sol in solve(A + B, x)]
sol_bc = [simplify(sol) for sol in solve(B + C, x)]
sol_ac = [simplify(sol) for sol in solve(A + C, x)]

# Remove extraneous root 0 that shows up - bc it never occurs in this problem formulation
# Helper function to remove zeros
def remove_zeros(sol_list):
    while 0 in sol_list:
        sol_list.remove(0)
remove_zeros(sol_ab)
remove_zeros(sol_bc)
remove_zeros(sol_ac)

# Check dominant balances to see if roots belong to small or large epsilon regimes
AB_valid_small_eps = (abs(A.subs(x, sol_ab[0]).subs(epsilon, 0.0001)) > abs(C.subs(x, sol_ab[0]).subs(epsilon, 0.001))) # A,B >> C small eps
AB_valid_large_eps = (abs(A.subs(x, sol_ab[0]).subs(epsilon, 10000)) > abs(C.subs(x, sol_ab[0]).subs(epsilon, 1000))) # A,B >> C large eps

BC_valid_small_eps = (abs(B.subs(x, sol_bc[0]).subs(epsilon, 0.0001)) > abs(A.subs(x, sol_bc[0]).subs(epsilon, 0.001))) # B,C >> A small eps
BC_valid_large_eps = (abs(B.subs(x, sol_bc[0]).subs(epsilon, 10000)) > abs(A.subs(x, sol_bc[0]).subs(epsilon, 1000))) # B,C >> A large eps

AC_valid_small_eps = (abs(A.subs(x, sol_ac[0]).subs(epsilon, 0.0001)) > abs(B.subs(x, sol_ac[0]).subs(epsilon, 0.001))) # A,C >> B small eps
AC_valid_large_eps = (abs(A.subs(x, sol_ac[0]).subs(epsilon, 10000)) > abs(B.subs(x, sol_ac[0]).subs(epsilon, 1000))) # A,C >> B large eps

print("AB valid for small epsilon:", AB_valid_small_eps)
print("AB valid for large epsilon:", AB_valid_large_eps)
print("BC valid for small epsilon:", BC_valid_small_eps)
print("BC valid for large epsilon:", BC_valid_large_eps)
print("AC valid for small epsilon:", AC_valid_small_eps)
print("AC valid for large epsilon:", AC_valid_large_eps)
print("\n")

# TODO (optional): Figure out how to go backwards from multiple complex roots/roots of unity to simpler format (e.g. 1/eps**5)

# Display results
print("polynomial:")
display(polynomial)
print("\n")

print("Balance A, B | ", len(sol_ab), "roots")
display(sol_ab)
print("\n")

print("Balance B, C | ", len(sol_bc), "roots")
display(sol_bc)
print("\n")

print("Balance A, C | ", len(sol_ac), "roots")
display(sol_ac)
print("\n")

AB valid for small epsilon: True
AB valid for large epsilon: False
BC valid for small epsilon: True
BC valid for large epsilon: False
AC valid for small epsilon: False
AC valid for large epsilon: True


polynomial:


epsilon*x**9 + x**7 + 1



Balance A, B |  2 roots


[-sqrt(-1/epsilon), sqrt(-1/epsilon)]



Balance B, C |  7 roots


[exp(I*pi/7),
 -1,
 -cos(2*pi/7) - I*cos(3*pi/14),
 sin(pi/14) + I*cos(pi/14),
 sin(5*pi/14) - I*cos(5*pi/14),
 -sin(3*pi/14) + I*cos(3*pi/14),
 sin(pi/14) - I*cos(pi/14)]



Balance A, C |  9 roots


[(-1/epsilon)**(1/9),
 (-1/epsilon)**(1/9)*(-1 - sqrt(3)*I)/2,
 (-1/epsilon)**(1/9)*(-1 + sqrt(3)*I)/2,
 -(-1/epsilon)**(1/9)*exp(I*pi/9),
 -(-1/epsilon)**(1/9)*exp(-I*pi/9),
 (-1/epsilon)**(1/9)*exp(-2*I*pi/9),
 (-1/epsilon)**(1/9)*exp(2*I*pi/9),
 (-1/epsilon)**(1/9)*(sin(pi/18) - I*cos(pi/18)),
 (-1/epsilon)**(1/9)*(sin(pi/18) + I*cos(pi/18))]

In [ ]:
sol_ac_noeps = []
for i in range(len(sol_ac)):
  sol_ac_noeps.append(sol_ac[i].subs(epsilon,1))

In [ ]:
# Convert each root to its polar form and then express it using Euler's formula
euler_forms = []
for root in sol_ac_noeps:
    magnitude, angle = cmath.polar(root)
    euler_form = f"{magnitude} * exp(i * {angle})"
    euler_forms.append(euler_form)
euler_forms

['1.0 * exp(i * 0.4487989505128276)',
 '1.0 * exp(i * -2.243994752564138)',
 '1.0 * exp(i * 3.141592653589793)',
 '1.0 * exp(i * -0.4487989505128276)',
 '1.0 * exp(i * 1.3463968515384828)',
 '1.0 * exp(i * -1.3463968515384828)',
 '1.0 * exp(i * 2.243994752564138)']

### Function to generate latex problem statement

In [ ]:
\noindent\hrulefill
problem = "Solve for analytical approximaitions to the roots of the following polynomial. Present the roots for both large and small \epsilon."
latex(polynomial)

'\\epsilon x^{9} - x^{4} + 1'

In [ ]:
# Define the file name
file_name = "problem_dataset.tex"

# Write to file
with open(file_name, "w") as file:
    file.write("\\documentclass{article}\n")
    file.write("\\usepackage{amsmath}\n")
    file.write("\\begin{document}\n")
    file.write("\\section*{Quadratic Equation}\n")
    file.write("The quadratic equation is given by:\n")
    file.write("\\[" + latex(polynomial) + "\\]\n")
    file.write("\\end{document}")

### Helper function to generate sympy polynomial

In [ ]:
# Generates sympy polynomial from
def polynomial_from_coefficients(coefficients):
    """
    Converts a list of coefficients into a polynomial expression using sympy.

    Parameters:
    coefficients (list): A list of coefficients, where the index represents the power of x.

    Returns:
    sympy expression: The polynomial expression.
    """
    x = symbols('x')
    coefficients_reverse = coefficients
    coefficients_reverse.reverse()
    polynomial = sum(coef * x**i for i, coef in enumerate(coefficients_reverse))
    return Poly(polynomial)

In [ ]:
# Test functions
coefficients = generate_polynomial(degree=10, num_terms=3, coeff_bounds=[-10,10])
print("Coefficients", coefficients)

polynomial_expression = polynomial_from_coefficients(coefficients)
polynomial_expression

Coefficients [0, 0, -8, 0, 5, 0, 0, 0, 0, 0, 7]


Poly(-8*x**8 + 5*x**6 + 7, x, domain='ZZ')

## Generate latex for problem and solution

## Check that analytical solutions are approx. equal to the numerical solution.

If valid, generate latex -> add to dataset
- Check for duplicates??

Discard problem otherwise

## Unit tests for each function

### Generate polynomial
- Generate several polynomials, check that there are exactly n nonzero coefficients and that there is a constant.

### Nondimensionalize polynomial
- Generate many polynomials, nondimensionalize, compare numerically-solved roots to check they are the same (but scaled)????????????

In [ ]:
def test_generate(max_degree, num_terms, coeff_bounds):
  # Test 100 different runs of the function
  for i in range(100):
    # Solve
    coeffs = generate_polynomial(max_degree=max_degree, num_terms=num_terms, coeff_bounds=coeff_bounds)
    # Check length of array
    assert len(coeffs) == max_degree + 1
    # Check number of terms
    assert len([x for x in coeffs if x != 0]) == num_terms
    # Check for nonzero constant term
    assert coeffs[-1] != 0
    # Check that coeff bounds are correct
    all(coeff_bounds[0] <= x <= coeff_bounds[1] for x in coeffs)
  print("All tests passed! (Generate polynomial function)")

In [ ]:
# Parameters
max_degree = 10
num_terms = 3
coeff_bounds=[-10,10]

# Run tests
test_generate(max_degree, num_terms, coeff_bounds)

All tests passed! (Generate polynomial function)
